# Practical Exercise 4.05: Cash Flow Matching

In [ ]:
import numpy as np
import pandas as pd

# Bond cash flows

bond_flows = {
    'A': [101.5, 0, 0, 0, 0],
    'B': [8.75, 508.75, 0, 0, 0],
    'C': [20, 20, 1020, 0, 0],
    'D': [25, 25, 25, 1025, 0],
    'E': [13.75, 13.75, 13.75, 13.75, 513.75]
}

# Cash flows from liabilities

loan_flows = [450000, 450000, 450000, 450000, 15450000]

# Initialize the number of b required

bonds_needed = {bond: 0 for bond in bond_flows.keys()}

# Function to match liability cash flows with bond cash flows

def cover_flows(bond, year, bonds_needed, loan_flows):
    num_bonds = loan_flows[year] / bond_flows[bond][year]
    bonds_needed[bond] += num_bonds
    for y in range(5):
        loan_flows[y] -= num_bonds * bond_flows[bond][y]
    return bonds_needed, loan_flows

# Cover liability cash flows starting with the longest maturity

for bond, year in zip(['E', 'D', 'C', 'B', 'A'], range(4, -1, -1)):
    bonds_needed, loan_flows = cover_flows(bond, year, bonds_needed, loan_flows)

# Round up the amounts of bonds needed

bonds_needed = {bond: round(num,0) for bond, num in bonds_needed.items()}

# Calculate the cash flows generated

generated_flows = [
    round(sum(num * bond_flows[bond][0] for bond, num in bonds_needed.items())),
    round(sum(num * bond_flows[bond][1] for bond, num in bonds_needed.items())),
    round(sum(num * bond_flows[bond][2] for bond, num in bonds_needed.items())),
    round(sum(num * bond_flows[bond][3] for bond, num in bonds_needed.items())),
    round(sum(num * bond_flows[bond][4] for bond, num in bonds_needed.items()))
]

# Show results

print("Bonds required:", bonds_needed)
print("Cash flows generated:", generated_flows)
print("Outstanding balances:", loan_flows)


Portfolio cost and surplus

In [ ]:
import pandas as pd

# Bond data
bond_yields = {
    'A': 0.0155,
    'B': 0.02,
    'C': 0.0225,
    'D': 0.0275,
    'E': 0.0315
}

bond_flows = {
    'A': [101.5, 0, 0, 0, 0],
    'B': [8.75, 508.75, 0, 0, 0],
    'C': [20, 20, 1020, 0, 0],
    'D': [25, 25, 25, 1025, 0],
    'E': [13.75, 13.75, 13.75, 13.75, 513.75]
}

bonds_needed = {'A': 338.0, 'B': 69.0, 'C': 35.0, 'D': 36.0, 'E': 30073.0}
loan_amount = 15000000  # Amount of the loan



# Function to calculate the price of a bond
def calculate_bond_price(flows, yield_to_maturity):
    return sum(flow / (1 + yield_to_maturity) ** (t + 1) for t, flow in enumerate(flows))

# Calculating bond prices
bond_prices = {bond: calculate_bond_price(flows, bond_yields[bond]) for bond, flows in bond_flows.items()}

# Create the DataFrame
df_bond_details = pd.DataFrame({
    'Bond': bonds_needed.keys(),
    'Quantity': bonds_needed.values(),
    'Price': [bond_prices[bond] for bond in bonds_needed],
    'Total Cost': [bonds_needed[bond] * bond_prices[bond] for bond in bonds_needed]
})

# Add total sum row without values in Quantity and Price
df_bond_details.loc[len(df_bond_details)] = ['Total', '', '', df_bond_details['Total Cost'].sum()]

# Format the columns
df_bond_details['Quantity'] = df_bond_details['Quantity'].apply(lambda x: f"{x:,.2f}" if x != '' else '')
df_bond_details['Price'] = df_bond_details['Price'].apply(lambda x: f"{x:,.2f}" if x != '' else '')
df_bond_details['Total Cost'] = df_bond_details['Total Cost'].apply(lambda x: f"{x:,.2f}")

# Calculate the difference with the loan amount
total_cost_sum = df_bond_details.loc[df_bond_details['Bond'] == 'Total', 'Total Cost'].replace({',': ''}, regex=True).astype(float).values[0]
difference = loan_amount-total_cost_sum

# Add the difference to the DataFrame
df_bond_details['Loan surplus'] = ''
df_bond_details.loc[df_bond_details['Bond'] == 'Total', 'Loan surplus'] = f"{difference:,.2f}"

# Reset index and hide index column
df_bond_details.reset_index(drop=True, inplace=True)
df_bond_details.style.hide(axis='index')
